In [6]:
from collections import defaultdict

# Function to compute FIRST set for a given non-terminal
def compute_first(non_terminal):
    if non_terminal in first_sets and first_sets[non_terminal]:  # Prevent recomputation
        return

    first_set = set()
    
    for rule in grammar[non_terminal]:
        if rule == '#':  # Handling epsilon explicitly
            first_set.add('#')
            continue

        for symbol in rule:
            if symbol.islower() or not symbol.isalpha():  # Terminal
                first_set.add(symbol)
                break
            else:  # Non-terminal
                compute_first(symbol)
                first_set.update(first_sets[symbol] - {'#'})  # Add FIRST(symbol) excluding epsilon
                
                if '#' not in first_sets[symbol]:  # Stop if ε not in FIRST
                    break
        else:
            first_set.add('#')  # If all symbols derive ε, add ε to FIRST

    first_sets[non_terminal] = first_set

# Function to compute FOLLOW set for a given non-terminal
def compute_follow(non_terminal):
    if non_terminal in follow_sets and follow_sets[non_terminal]:  # Prevent recomputation
        return

    follow_set = set()
    if non_terminal == start_symbol:  # Start symbol gets $
        follow_set.add('$')

    for rule in grammar:
        for production in grammar[rule]:
            if non_terminal in production:
                index = production.index(non_terminal)

                while index < len(production) - 1:  # Check next symbols
                    next_symbol = production[index + 1]

                    if next_symbol.isupper():  # If non-terminal
                        follow_set.update(first_sets[next_symbol] - {'#'})

                        if '#' in first_sets[next_symbol]:  # If nullable, continue checking
                            index += 1
                        else:
                            break
                    else:  # Terminal
                        follow_set.add(next_symbol)
                        break
                else:  # If reached the end of production
                    if rule != non_terminal:  # Avoid self-dependency
                        compute_follow(rule)
                        follow_set.update(follow_sets[rule])

    follow_sets[non_terminal] = follow_set

# Function to construct the LL(1) parsing table
def construct_ll1_table():
    parsing_table = defaultdict(lambda: defaultdict(set))

    for non_terminal in grammar:
        for production in grammar[non_terminal]:
            first_set = set()

            if production == '#':  # If epsilon production
                first_set = follow_sets[non_terminal]
            else:
                for symbol in production:
                    if symbol.islower() or not symbol.isalpha():  # Terminal
                        first_set.add(symbol)
                        break
                    else:  # Non-terminal
                        first_set.update(first_sets[symbol] - {'#'})

                        if '#' not in first_sets[symbol]:
                            break
                else:
                    first_set.add('#')  # All nullable symbols lead to epsilon

            for terminal in first_set:
                if terminal != '#':
                    parsing_table[non_terminal][terminal].add(production)

            if '#' in first_set:  # Add FOLLOW(non_terminal) if epsilon is in FIRST
                for terminal in follow_sets[non_terminal]:
                    parsing_table[non_terminal][terminal].add(production)

    return parsing_table


# Input grammar and setup
grammar = defaultdict(list)
first_sets = defaultdict(set)  # Using defaultdict to avoid KeyErrors
follow_sets = defaultdict(set)

# Take grammar input
num_rules = int(input("Enter number of production rules: "))
print("Enter grammar rules (one per line, format: A -> B|C):")
start_symbol = input("Enter the start symbol: ").strip()

# Read production rules
for _ in range(num_rules):
    line = input().strip()

    if "->" not in line:
        print(f"Invalid input format: '{line}'. Skipping this rule.")
        continue

    non_terminal, rhs = line.split("->")
    productions = rhs.split("|")
    grammar[non_terminal.strip()] = [prod.strip() for prod in productions]

# Compute FIRST and FOLLOW sets
for non_terminal in grammar:
    compute_first(non_terminal)

for non_terminal in grammar:
    compute_follow(non_terminal)

# Display FIRST sets
print("\nFIRST sets:")
for non_terminal, first_set in first_sets.items():
    print(f"FIRST({non_terminal}) = {{ {', '.join(sorted(first_set))} }}")

# Display FOLLOW sets
print("\nFOLLOW sets:")
for non_terminal, follow_set in follow_sets.items():
    print(f"FOLLOW({non_terminal}) = {{ {', '.join(sorted(follow_set))} }}")

# Construct LL(1) Parsing Table
parsing_table = construct_ll1_table()

# Display LL(1) Parsing Table
print("\nLL(1) Parsing Table:")
for non_terminal in parsing_table:
    print(f"{non_terminal} ->", end=" ")
    for terminal in parsing_table[non_terminal]:
        print(f"{terminal}: {', '.join(parsing_table[non_terminal][terminal])}", end=" | ")
    print()


Enter number of production rules:  4


Enter grammar rules (one per line, format: A -> B|C):


Enter the start symbol:  S
 S -> ABC|C
 A->a|bB|#
 B->p|#
 C->c



FIRST sets:
FIRST(A) = { #, a, b }
FIRST(B) = { #, p }
FIRST(C) = { c }
FIRST(S) = { a, b, c, p }

FOLLOW sets:
FOLLOW(S) = { $ }
FOLLOW(A) = { c, p }
FOLLOW(B) = { c, p }
FOLLOW(C) = { $ }

LL(1) Parsing Table:
S -> a: ABC | c: ABC, C | b: ABC | p: ABC | 
A -> a: a | b: bB | c: # | p: # | 
B -> p: p, # | c: # | 
C -> c: c | 
